In [1]:
#this script tries to improve the iv analysis
#trying to correct the jump/unwrap issue when Ites changes to fast with the current Ibias resoluation

In [4]:
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/matt/readout/host/')
import ali_offline_demod as dm
import imp
import numpy as np
imp.reload(dm)
from scipy.signal import savgol_filter, find_peaks

/tmp/ipykernel_1147980/2354833800.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [5]:
path = 'improve_iv_test_data/'

demod=np.loadtxt(path+'demod.txt')
Ibias=np.loadtxt(path+'bias_data_20240201160810.txt')
t_demod=np.loadtxt(path+'t_demod.txt')




In [6]:
%matplotlib qt
for ch in range(demod.shape[0]):
    plt.plot(demod[ch])

In [7]:
def get_mean_current(bias_info,times,data):
    """
    get the mean of Ites for all channels based on bias values and times
    time is unix time
    data is the output dm.demodulate, in unit of phi0 numbers
    """
    data_bin=np.zeros((data.shape[0],bias_info.shape[0]))
    print (data_bin.shape)
    for j in range(data.shape[0]):
        data_ch=data[j]
        for i in range(bias_info.shape[0]):
            bs_time=bias_info[i,0]
            chunck=data_ch[np.where((times>bs_time+0.4)&(times<bs_time+1.6))]
            data_bin[j,i]=np.mean(chunck)
    data_bin=np.apply_along_axis(unwrap_change_current_per_chan, 1, data_bin)
    return data_bin 

def unwrap_change_current_per_chan(data_ch):
    data_ch_unwrap=np.unwrap(data_ch,discont=0.48,period=1)
    data_ch_uA=data_ch_unwrap*9
    return data_ch_uA

In [8]:
data_demods_bin = get_mean_current(Ibias,t_demod[0],demod/9)
print(data_demods_bin)

(235, 187)
[[ 1.34189561  1.269452    1.3742307  ...  0.71600167  0.84121438
   0.81833873]
 [-1.04930274 -1.02349154 -0.98450433 ... -1.20533495 -1.13165521
  -1.13291144]
 [ 0.45861181  0.4822657   0.51014485 ...  0.41317694  0.49743002
   0.40071808]
 ...
 [ 0.36450675  0.39996255  0.42018264 ...  0.31914634  0.41055899
   0.30550218]
 [ 2.59153268  2.62448729  2.65722618 ...  2.5426435   2.60200803
   2.54477767]
 [ 1.25305558  1.28252024  1.29280133 ...  1.24326018  1.27817652
   1.1708859 ]]


In [9]:
%matplotlib qt
for ch in range(data_demods_bin.shape[0]):
    if np.max(data_demods_bin[ch])-np.min(data_demods_bin[ch])<20:continue
    plt.plot(Ibias[:,1],data_demods_bin[ch],'b', alpha=0.1)
    smooth=savgol_filter(data_demods_bin[ch], data_demods_bin[ch].shape[0], 10)
    plt.plot(Ibias[:,1],smooth,'r', alpha=0.1)
    dd=np.diff(np.diff(smooth))
    ind_min=np.argmin(dd)+2
    peaks_nb,_=find_peaks(0-data_demods_bin[ch],width=10)
    #plt.vlines(Ibias[ind_max,1], -200,100,'orange')
    if len(peaks_nb)>0 and np.max(data_demods_bin[ch])-np.min(data_demods_bin[ch])>20:
        plt.vlines(Ibias[ind_min,1], -200,100,'green')
        plt.vlines(Ibias[peaks_nb[0],1], -200,100,'orange')

In [10]:
%matplotlib qt
for ch in range(data_demods_bin.shape[0]):
    plt.plot(data_demods_bin[ch],'b', alpha=0.3)
    plt.plot(np.diff(data_demods_bin[ch])*10,'r', alpha=0.3)

In [11]:
def IV_correction(resps):
    """
    This function smooth the IV curve and then find the normal and superconducting point
    it will then try to correct the jump caused by unwrapping large Ites changes with current Ibias resolution
    """ 
    #only getting Al TES normal point
    peaks_nb,_=find_peaks(0-resps,width=20)
    smooth=savgol_filter(data_demods_bin[ch], data_demods_bin[ch].shape[0], 10)
    dd=np.diff(np.diff(smooth))
    ind_sc=np.argmin(dd)+2
    if len(peaks_nb)==0 or peaks_nb[0]<30:
        return np.zeros(resps.shape[0])
    peak_nb=peaks_nb[0]
    #here we know that before peaks_nb[0] and after ind_sc the Ites is monotonic 
    resps_nb=resps[2:int(peak_nb-10)]
    resps_sc=resps[int(ind_sc+10):-10]
    resps_cor=np.zeros(resps.shape[0])
    resps_cor_acc=np.zeros(resps.shape[0])
    for i in range(resps.shape[0]):
        if i>=2 and i<peak_nb-10 and resps[i]-resps[i-1]>0.5:
            resps_cor[i]=-9
        if i>= ind_sc+10 and i<resps.shape[0]-5 and resps[i]-resps[i-1]>0.5:
            resps_cor[i]=-9
    for i in range(resps.shape[0]):
        if i>0:
            resps_cor_acc[i]=np.sum(resps_cor[:i+1])
    resps_corr=resps+resps_cor_acc
    return resps_corr, ind_sc
    
        
            
    
    
    

In [12]:
def IV_analysis_ch_duo(bias_currents,resps,Rsh=0.4,filter_Rn_Al=False,plot='None'):
    """
    This method correct the SC and normal region for jump phi0 issue, and then fit for TES parameters
    Outputs:
    dataframe containing timestream of Ites,Rtes,Vtes,Rn_almn,Rn_al
    """ 
    peaks_nb,_=find_peaks(0-resps,width=20)
    max_ites=np.max(resps)
    min_ites=np.min(resps)
    if len(peaks_nb)==0 or peaks_nb[0] < 20 or max_ites-min_ites<20:
        Rn_almn=np.nan
        Rn_al=np.nan
        Rtes=np.ones(bias_currents.shape[0])*np.nan
        Vtes=np.ones(bias_currents.shape[0])*np.nan
        Ites=np.ones(bias_currents.shape[0])*np.nan
        bps=np.ones(bias_currents.shape[0])*np.nan
        Pbias=np.ones(bias_currents.shape[0])*np.nan
    else:   
        peak_nb=peaks_nb[0]
        resps_correct,peak_sc=IV_correction(resps)
        resps_nb=resps_correct[2:int(peak_nb-10)]
        resps_sc=resps_correct[int(peak_sc+10):-10]
        bias_nb=bias_currents[2:peak_nb-10]
        bias_sc=bias_currents[int(peak_sc+10):-10]
        r_ratio_al,shift = np.polyfit(bias_nb, resps_nb, 1)
        r_ratio_almn,shift_sc = np.polyfit(bias_sc, resps_sc, 1)
        Rn_almn=Rsh/r_ratio_almn-Rsh*1e-3 #Ohm
        Rn_al=Rsh/r_ratio_al-Rsh*1e-3-Rn_almn  #Ohm
        #print ('normal resistance for al TES',Rn_al)
        Ites=resps_correct-shift #uA
        Ishunt=bias_currents*1e-3-Ites*1e-6 #A
        Vshunt=Ishunt*Rsh*1e-3 #V
        Rtes=Vshunt/(Ites*1e-6)-Rn_almn #Ohm
        bps=Rtes/Rn_al
        Vtes=Rtes*Ites#uV
        Pbias=Ites**2*(Rtes+Rn_almn) #pW the overall heating caused by both Al and Almn

        
        if filter_Rn_Al == True:
            if Rn_al < 5e-3 or Rn_al > 20e-3:##: #filter out non-responding channels
                Rn_al=np.nan
                Rtes=np.ones(bias_currents.shape[0])*np.nan
                Vtes=np.ones(bias_currents.shape[0])*np.nan
                Ites=np.ones(bias_currents.shape[0])*np.nan
                bps=np.ones(bias_currents.shape[0])*np.nan
        if plot=='org':
            plt.gcf().subplots_adjust(bottom=0.2)
            plt.gcf().subplots_adjust(left=0.2)
            plt.tick_params(axis='both', which='major', labelsize=16)
            plt.plot(bias_currents,Ites,alpha=0.8)
            plt.xlabel('$I_{bias}$ (mA)',fontsize=18)
            plt.ylabel('$I_{tes}$ (μA)',fontsize=18)


        if plot=='IV':
            plt.gcf().subplots_adjust(bottom=0.2)
            plt.gcf().subplots_adjust(left=0.2)
            plt.tick_params(axis='both', which='major', labelsize=16)
            plt.plot(Vtes,Ites,alpha=0.8)
            plt.xlabel('$V_{tes}$ (μV)',fontsize=18)
            plt.ylabel('$I_{tes}$ (μA)',fontsize=18)
        if plot=='bp':
            plt.gcf().subplots_adjust(bottom=0.2)
            plt.gcf().subplots_adjust(left=0.2)
            plt.tick_params(axis='both', which='major', labelsize=16)
            plt.plot(bias_currents,bps*100,alpha=0.3)
            plt.ylim(0,1.2*100)
            plt.ylabel('$\%R_n$',fontsize=18)
            plt.xlabel(r'$I_{bias}$',fontsize=18)
        if plot=='pbias':
            plt.gcf().subplots_adjust(bottom=0.2)
            plt.gcf().subplots_adjust(left=0.2)
            plt.tick_params(axis='both', which='major', labelsize=16)
            plt.plot(Pbias,Rtes*1000,alpha=0.3)
            plt.ylim(0,15)
            plt.xlim(200,900)
            plt.ylabel(r'$R_{tes}$',fontsize=18)
            plt.xlabel(r'$P_{bias}$',fontsize=18)
        print (Rn_almn)
        print (Rn_al)
    return Rn_almn,Rn_al,Rtes,Vtes,Ites,bps,Pbias

In [17]:
for i in range(data_demods_bin.shape[0]):
    IV_analysis_ch_duo(Ibias[:,1],data_demods_bin[i],Rsh=0.4,filter_Rn_Al=False,plot='org')
    

0.008236722765843309
0.012669464469843779
0.00839668894051447
0.012914254387262859
0.008455896352441086
0.013038526699845081
0.00829293080018696
0.012668645208914623
0.008316649122341282
0.012721706518709418
0.008596796901090978
0.013140038308610538
0.008207112637757016
0.01258238015695947
0.008297741927963152
0.012767023772533845
0.008311572926299431
0.012687861665791203
0.008313324026267792
0.012684479697122544
0.008124007068523073
0.012647640883599183
0.008182112845207397
0.012825647330235989
0.008569295455678725
0.013358736000843124
0.008391624693572821
0.01318110654598563
0.008325067596100378
0.013047975683880561
0.008148999117282148
0.012745855251527499
0.008226681081250577
0.01292323815930906
0.00822720923651275
0.012900932056672008
0.008414327282759197
0.013189281073765446


In [16]:
for i in range(data_demods_bin.shape[0]):
    data_ch_cor=IV_correction(data_demods_bin[i])
    plt.plot(data_demods_bin[i],'b',alpha=0.3)

In [15]:
data_demods_bin.shape

(235, 187)